In [1]:
import pandas as pd 
from pypath.utils import mapping
import numpy as np 

In [2]:
t1k = pd.read_csv('/mnt/c/Users/natha/Documents/tkgdti_top1000_results.csv')
t1k = t1k[lambda x: ~x.in_targetome_extended]
t1k = t1k[['inhibitor', 'target', 'gnn_drug_rank', 'cpx_drug_rank']]
t1k = t1k.assign(avg_drug_rank = lambda x: (x.gnn_drug_rank + x.cpx_drug_rank)/ 2)
t1k.head() 

,inhibitor,target,gnn_drug_rank,cpx_drug_rank,avg_drug_rank
0,PD153035,ERBB3,1.0,1.0,1.0
4,Ibrutinib (PCI-32765),ERBB3,1.0,2.0,1.5
5,BMS-754807,PRKAA2,1.0,6.0,3.5
6,SU14813,FLT1,1.0,1.0,1.0
9,Venetoclax,BAX,1.0,5.0,3.0


In [3]:
tge = pd.read_csv('../../../local/data/targetome_extended-01-23-25.csv')
tge_drugs = pd.read_csv('../../../local/data/targetome_extended_drugs-01-23-25.csv')[['inchi_key', 'drug_name', 'clinical_phase']]

status = [] 
for i in tge_drugs.clinical_phase.values:
    if i == 'Approved': 
        status.append('Approved')
    elif i in ['Phase 1 Clinical Trials', 'Phase 3 Clinical Trials', 'Phase 2 Clinical Trials', 'Early Phase 1 Clinical Trials']:
        status.append('in_trials')
    elif i in ['preclinical compounds with bioactivity data']:
        status.append('preclinical')
    else:
        status.append('unknown')

tge_drugs = tge_drugs.assign(status = status)
tge = tge.merge(tge_drugs, on='inchi_key', how='left')
tge = tge[lambda x:( x.assay_type.isin(['Ki', 'Kd', 'IC50'])) & (x.assay_relation.isin(['<', '=', '<='])) & (x.assay_value < 1000)]
tge = tge[['uniprot_id', 'drug_name', 'status']]

uniprots = tge.uniprot_id.unique()
genesymbs = [] 
for i,uniprot in enumerate(uniprots):
    print(f'mapping uniprot -> genesymbol [{i}/{len(uniprots)}]', end='\r')
    gs = mapping.map_name(uniprot, 'uniprot', 'genesymbol') 
    if len(gs) > 0:
        genesymbs.append(gs.pop())
    else:
        genesymbs.append(None)

uni2gene = pd.DataFrame({'uniprot_id': uniprots, 'genesymbol': genesymbs})

tge = tge.merge(uni2gene, on='uniprot_id', how='left')
tge = tge.dropna() 
tge = tge.drop_duplicates()

drug_names = np.sort(tge.drug_name.unique()).tolist()
idx2drug = pd.DataFrame({'drug_idx': np.arange(len(drug_names)).astype(str), 'drug_name':drug_names})
tge = tge.merge(idx2drug, on='drug_name', how='left')

tge.head() 


/tmp/ipykernel_24509/3408380080.py:2: DtypeWarning: Columns (10,11,13,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  tge_drugs = pd.read_csv('../../../local/data/targetome_extended_drugs-01-23-25.csv')[['inchi_key', 'drug_name', 'clinical_phase']]


,uniprot_id,drug_name,status,genesymbol,drug_idx
0,Q9GZT9,Alpha-ketoglutaric Acid,preclinical,EGLN1,227973
1,P00918,Protocatechuic Acid,preclinical,CA2,306413
2,Q9ULX7,Protocatechuic Acid,preclinical,CA14,306413
3,A8MPY1,Gamma-aminobutyric Acid,in_trials,GABRR3,241275
4,O00591,Gamma-aminobutyric Acid,in_trials,GABRP,241275


In [4]:
lib25 = pd.read_csv('/mnt/c/Users/natha/Downloads/LIB250404AA_results_for_annotating.csv')
lib25.head() 
# "C:\Users\natha\Downloads\LIB250404AA_results_for_annotating.csv" 

,Gene,contrasts,total,not_filtered,num_sig,min_qvalue,min_abslfc,lfc_min,lfc_mean,lfc_max,...,num_down,tiers,mid_lfc,mid_pval,sig_mid_lfc,sig_mid_pval,camera_dir,camera_pval,camera_qval,depmap_fitness_summary
0,A2ML1,HS5_VenAzavsHS5_DMSO_D14,5,4,2,2.171861e-26,1.204285,-2.021473,-1.612879,-1.204285,...,2,Tier3,-1.204285,4.661675e-02,-2.021473,2.669653e-27,Down,0.208254,0.999933,Neither
1,A4GALT,HS5_VenAzavsVenAza_D14,5,4,1,2.200741e-02,2.497819,-2.497819,-2.497819,-2.497819,...,1,Singleton,1.163028,3.498219e-02,-2.497819,1.139836e-02,Down,0.580238,0.999794,Neither
2,AAAS,HS5_VenAzavsVenAza_D14,5,2,1,8.485449e-04,3.617198,-3.617198,-3.617198,-3.617198,...,1,Singleton,-3.617198,2.029384e-04,-3.617198,2.029384e-04,Down,0.308154,0.999794,Neither
3,AADACL2,VenAzavsDMSO_D14,5,3,2,3.408625e-43,3.014155,-5.122525,-4.068340,-3.014155,...,2,Tier2,-5.122525,1.171016e-09,-3.014155,4.759723e-45,Down,0.003746,0.329625,Neither
4,AADACL3,HS5_VenAzavsVenAza_D14,5,4,1,1.765651e-03,5.983106,-5.983106,-5.983106,-5.983106,...,1,Singleton,-0.535286,1.543735e-01,-5.983106,4.908400e-04,Down,0.382781,0.999794,Neither


In [5]:
clin_drugs = tge[lambda x: x.status == 'Approved'].groupby('genesymbol').agg({'drug_idx':lambda x: ';'.join(x)}).reset_index().rename(columns={'drug_idx':'targetome[clinical]'}).rename(columns={'genesymbol':'Gene'})
preclin_drugs = tge[lambda x: x.status == 'preclinical'].groupby('genesymbol').agg({'drug_idx':lambda x: ';'.join(x)}).reset_index().rename(columns={'drug_idx':'targetome[preclinical]'}).rename(columns={'genesymbol':'Gene'})
trials_drugs = tge[lambda x: x.status == 'in_trials'].groupby('genesymbol').agg({'drug_idx':lambda x: ';'.join(x)}).reset_index().rename(columns={'drug_idx':'targetome[in_trials]'}).rename(columns={'genesymbol':'Gene'})
unknown_drugs = tge[lambda x: x.status == 'unknown'].groupby('genesymbol').agg({'drug_idx':lambda x: ';'.join(x)}).reset_index().rename(columns={'drug_idx':'targetome[unknown]'}).rename(columns={'genesymbol':'Gene'})

lib25 = lib25.merge(clin_drugs, on='Gene', how='left')
lib25 = lib25.merge(preclin_drugs, on='Gene', how='left')
lib25 = lib25.merge(trials_drugs, on='Gene', how='left')
lib25 = lib25.merge(unknown_drugs, on='Gene', how='left')


In [6]:

# modify this so that we only select the  top 5 most likely drugs (lowest ranks)
pred_drugs = t1k.groupby('target').apply(lambda x: x.sort_values('avg_drug_rank').head(5)).reset_index(drop=True).groupby('target').agg({'inhibitor':lambda x: ';'.join(x)}).reset_index().rename(columns={'inhibitor':'predicted[top5;rank<1000]', 'target':'Gene'})

# select the inhbitor with the lowest drug rank (gnn_drug_rank) for each target
pred_best_drugs = t1k.groupby('target').apply(lambda x: x.sort_values('gnn_drug_rank').iloc[0]).reset_index(drop=True).rename(columns={'inhibitor':'predicted[most-likely]', 'target':'Gene', 'gnn_drug_rank':'predicted[most-likely-rank]'})
pred_best_drugs = pred_best_drugs[['Gene', 'predicted[most-likely]', 'predicted[most-likely-rank]']]

lib25 = lib25.merge(pred_drugs, on='Gene', how='left')
lib25 = lib25.merge(pred_best_drugs, on='Gene', how='left')
lib25.head().style.hide()

/tmp/ipykernel_24509/1871108298.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pred_drugs = t1k.groupby('target').apply(lambda x: x.sort_values('avg_drug_rank').head(5)).reset_index(drop=True).groupby('target').agg({'inhibitor':lambda x: ';'.join(x)}).reset_index().rename(columns={'inhibitor':'predicted[top5;rank<1000]', 'target':'Gene'})
/tmp/ipykernel_24509/1871108298.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to sile

Gene,contrasts,total,not_filtered,num_sig,min_qvalue,min_abslfc,lfc_min,lfc_mean,lfc_max,num_up,num_down,tiers,mid_lfc,mid_pval,sig_mid_lfc,sig_mid_pval,camera_dir,camera_pval,camera_qval,depmap_fitness_summary,targetome[clinical],targetome[preclinical],targetome[in_trials],targetome[unknown],predicted[top5;rank<1000],predicted[most-likely],predicted[most-likely-rank]
A2ML1,HS5_VenAzavsHS5_DMSO_D14,5,4,2,0.000000,1.204285,-2.021473,-1.612879,-1.204285,0,2,Tier3,-1.204285,0.046617,-2.021473,0.000000,Down,0.208254,0.999933,Neither,nan,nan,nan,nan,nan,nan,nan
A4GALT,HS5_VenAzavsVenAza_D14,5,4,1,0.022007,2.497819,-2.497819,-2.497819,-2.497819,0,1,Singleton,1.163028,0.034982,-2.497819,0.011398,Down,0.580238,0.999794,Neither,nan,nan,nan,nan,nan,nan,nan
AAAS,HS5_VenAzavsVenAza_D14,5,2,1,0.000849,3.617198,-3.617198,-3.617198,-3.617198,0,1,Singleton,-3.617198,0.000203,-3.617198,0.000203,Down,0.308154,0.999794,Neither,nan,nan,nan,nan,Bortezomib (Velcade);CHIR-99021;Sunitinib;Vismodegib (GDC-0449);Entrectinib,Bortezomib (Velcade),607.000000
AADACL2,VenAzavsDMSO_D14,5,3,2,0.000000,3.014155,-5.122525,-4.068340,-3.014155,0,2,Tier2,-5.122525,0.000000,-3.014155,0.000000,Down,0.003746,0.329625,Neither,nan,nan,nan,nan,nan,nan,nan
AADACL3,HS5_VenAzavsVenAza_D14,5,4,1,0.001766,5.983106,-5.983106,-5.983106,-5.983106,0,1,Singleton,-0.535286,0.154373,-5.983106,0.000491,Down,0.382781,0.999794,Neither,nan,nan,nan,nan,nan,nan,nan


# Data Dictionary for added columns 

Column: 
- `targetome[clinical]`: Drugs that have been FDA Approved and that have experimental evidence supporting a Kd, Ki, or IC50 value of less than 1000nM for the given target (column: `Gene`). Drugs are represented as integers that index drug names in `drug_indices.csv`.
- `targetome[preclinical]`: Drugs that have not entered FDA clinical trials and that have experimental evidence supporting a Kd, Ki, or IC50 value of less than 1000nM for the given target (column: `Gene`). Drugs are represented as integers that index drug names in `drug_indices.csv`.
- `targetome[in_trials]`: Drugs that have are in clinical trials and that have experimental evidence supporting a Kd, Ki, or IC50 value of less than 1000nM for the given target (column: `Gene`). Drugs are represented as integers that index drug names in `drug_indices.csv`.
- `targetome[unknown]`: Drugs whose status is unknown and that have experimental evidence supporting a Kd, Ki, or IC50 value of less than 1000nM for the given target (column: `Gene`). Drugs are represented as integers that index drug names in `drug_indices.csv`.
- `predicted[top5;rank<1000]`: The top-5 most likely drugs (lowest ranks) that were predicted by the `TKG-DTI` model to have a within-drug rank less than 1000 (Note: Using `GNN` model predictions)
- `predicted[most-likely]`: The most likely predicted drug-target (from the `GNN` model). 
- `predicted[most-likely-rank]`: The most likely predicted drug-target rank (from the `GNN` model). 


In [7]:
mds_wgcna = pd.read_excel('/mnt/c/Users/natha/Downloads/mds_wgcna_mm_w_myid.xlsx', sheet_name='module_membership')

mds_wgcna = mds_wgcna.merge(clin_drugs, left_on='gene_id', right_on='Gene', how='left').drop(columns=['Gene'])
mds_wgcna = mds_wgcna.merge(preclin_drugs, left_on='gene_id', right_on='Gene', how='left').drop(columns=['Gene'])
mds_wgcna = mds_wgcna.merge(trials_drugs, left_on='gene_id', right_on='Gene', how='left').drop(columns=['Gene'])
mds_wgcna = mds_wgcna.merge(unknown_drugs, left_on='gene_id', right_on='Gene', how='left').drop(columns=['Gene'])

mds_wgcna = mds_wgcna.merge(pred_drugs, left_on='gene_id', right_on='Gene', how='left').drop(columns=['Gene'])
mds_wgcna = mds_wgcna.merge(pred_best_drugs, left_on='gene_id', right_on='Gene', how='left').drop(columns=['Gene'])

mds_wgcna.head() 

,module,gene_id,myelo_id_gene_set,kme,depmap_core_fitness_pancan,depmap_core_fitness_haem_lymph,targetome[clinical],targetome[preclinical],targetome[in_trials],targetome[unknown],predicted[top5;rank<1000],predicted[most-likely],predicted[most-likely-rank]
0,Mod1 (turquoise),ITGAM,NaN,0.859171,no,no,NaN,NaN,NaN,NaN,ABT-737;Venetoclax;R547;PD153035;Bortezomib (V...,ABT-737,279.0
1,Mod1 (turquoise),FGR,NaN,0.849215,no,no,303900;236339;315460;231427;309409;228851;2507...,309103;286168;304763;300291;258963;297648;1813...,246335;311537;304830;309137;308516;307401;2287...,178023;305775;303907;305777,Cabozantinib;Tivozanib (AV-951);Entrectinib;Al...,Cabozantinib,11.0
2,Mod1 (turquoise),PYGL,NaN,0.849007,no,no,NaN,234942;234944;234943;58490;198271;185682;9790;...,227737,72927;199742;157262;14838;198247;276112;248169...,AST-487;JNJ-28312141;PP242,JNJ-28312141,788.0
3,Mod1 (turquoise),EMB,Promono-like,0.845061,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mod1 (turquoise),PADI4,NaN,0.841783,no,no,NaN,206650;217925;218023;246;319099;318731;318737;...,NaN,206665;45628,17-AAG (Tanespimycin);Panobinostat;Alisertib (...,Panobinostat,29.0


In [ ]:
mds_wgcna.to_csv('/mnt/c/Users/natha/Documents/dti-annotation-08-01-2025_v3/mds_wgcna_mm_w_myid_dti-annotated.csv', index=False)
lib25.to_csv('/mnt/c/Users/natha/Documents/dti-annotation-08-01-2025_v3/LIB250404AA_results_for_annotating_dti-annotated.csv', index=False)
idx2drug.to_csv('/mnt/c/Users/natha/Documents/dti-annotation-08-01-2025_v3/drug_indices.csv', index=False)